May 2022, Claire Valva
## data processing file (BAM/SAM/energy cycle):
In this file I process some era5 data for later analysis, specifically decomposing into Reynolds average and eddy terms. The variables we are interested in looking at (using the definitions from Thomspon and Woodworth 2014), brackets denote Reynold's average, primes denote eddy comp:
- Southern Annular Mode (SAM): defined as the leading PCA component of zonal mean zonal wind (in the SH)
- Baroclinic Annular Mode (BAM): defined as the leading PCA component of EKE ($[u'^2 + v'^2]/2$) in the SH.
- vertical wave heat flux: $[w'T']$
- eddy momentum flux: $[u'v']$
- meridional wave heat flux: $[v'T']$
- zonal mean KE: $[u]^2/2$
- meridional EP flux: $-\rho \cos (\varphi) [u'v']$
    - we can use a reference density profile of $\rho = e^{-z/H}$ where $H = 6.8 km$ and is the scale height
- vertical EP flux: $f\rho a \cos (\varphi) [u'\theta']/[\theta]_z$
    - we will use $6,371 km$ as the raidius of the earth
    - recall that the formula to calculate potential temperature is: $\theta = (p_0/p)^{R/c_p} T$, and we estimate $R/c_p = 0.286$.
- total heat flux: $c_p T + L_v q$ 
    - to estimate $L$ we will use the relation in chapter 18, appendix A of Vallis so that $L_v(T) = (2.501 \times 10^6 - 2359 T) J kg^{-1}$ where $T$ is in Celsius.
    - We will use $c_p =  1003 J kg^{-1} K^{-1}$
- CAPE
- will also want to look at the fourier components, is there always a sharp/"unique" time scale at ~25 days?


In [1]:
import os
import numpy as np
import xarray as xr

I borrowed some of this code from Dave, loading the data this way is much better than way I was originally doing. (Find the original file in this repo: https://github.com/dsconnelly/brewson).

The monthly means from era5 are 'ds633.1' but the full temporal data is found under file name 'ds633.0'.

In [2]:
# choose if want to use daily or monthly data (daily might take a prohibitively long time...)
monthly = True
if monthly:
    era_dir = '/gpfs/fs1/collections/rda/data/ds633.1/e5.moda.an.pl'
    era_dir_cape = '/gpfs/fs1/collections/rda/data/ds633.1/e5.moda.an.sfc/'
else:
    era_dir = '/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.an.pl'
    era_dir_cape = '/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.an.sfc'
    

In [3]:
years = [int(x) for x in sorted(os.listdir(era_dir))]

print(f'Found {len(years)} years of ERA5 data, starting with {years[0]} and ending with {years[-1]}.')

names = ['u', 'v', 'w', 'T', 'q']

ds_fnames = []
for year in years:
    year_dir = f'{era_dir}/{year}'
    fnames = [x for x in os.listdir(year_dir) if x.endswith('nc')]
    
    for name in names:
        fname = [x for x in fnames if f'_{name.lower()}.' in x][0]
        ds_fnames.append(f'{year_dir}/{fname}')
        
print(f'Found {len(ds_fnames)} files out of an expected {len(years) * len(names)}.')

# grab CAPE also
ds_fnames_cape = []
name = 'cape'
for year in years:
    year_dir = f'{era_dir_cape}/{year}'
    fnames_cape = [x for x in os.listdir(year_dir) if x.endswith('nc')]
    fname_cape = [x for x in fnames_cape if f'_{name.lower()}.' in x][0]
    ds_fnames.append(f'{year_dir}/{fname_cape}')

Found 43 years of ERA5 data, starting with 1979 and ending with 2021.
Found 215 files out of an expected 215.


In [13]:
with xr.open_mfdataset(ds_fnames, combine='by_coords') as ds:
    ds = ds.rename({
        'level' : 'z',
        'U' : 'u',
        'V' : 'v',
        'W' : 'w',
        'Q' : 'q',
        'CAPE' : 'cape'
    }).drop('utc_date')
    
    p = (100 * ds['z']).assign_attrs(units='Pa')
    lat = (np.pi * ds['latitude'] / 180).assign_attrs(units='radians_north')
    
    H, p_surf = 6800, p[-1]
    z = (-H * np.log(p / p_surf)).assign_attrs(units='meters')
    rho = np.exp(-z / H)
    
    ds['p'], ds['rho'] = p, rho
    ds = ds.assign_coords(z=z, latitude=lat)
    
    # reduce to daily means (unecessary if use monthly rather than daily)
    if not monthly:
        ds = ds.groupby("time.dayofyear").mean("time")
    
    display(ds)

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 516, z: 37)
Coordinates:
  * latitude   (latitude) float64 1.571 1.566 1.562 ... -1.562 -1.566 -1.571
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2021-12-01
  * z          (z) float64 4.697e+04 4.226e+04 3.95e+04 ... 348.8 172.2 -0.0
Data variables:
    cape       (time, latitude, longitude) float32 dask.array<shape=(516, 721, 1440), chunksize=(12, 721, 1440)>
    q          (time, z, latitude, longitude) float32 dask.array<shape=(516, 37, 721, 1440), chunksize=(12, 37, 721, 1440)>
    T          (time, z, latitude, longitude) float32 dask.array<shape=(516, 37, 721, 1440), chunksize=(12, 37, 721, 1440)>
    u          (time, z, latitude, longitude) float32 dask.array<shape=(516, 37, 721, 1440), chunksize=(12, 37, 721, 1440)>
    v          (time, z, latitude, longitude) float32 dask.array<shape=(516, 37, 721, 1440), c

In [14]:
# write down vars as arrays
p, lat, rho = ds['p'], ds['latitude'], ds['rho']
u, v, w = ds['u'], ds['v'], ds['w']
T, q = ds['T'], ds['q']
cape = ds['cape']

# compute potential temp:
theta = T * ((p_surf / p) ** 0.286)

In [15]:
# compute total heat flux
def comp_Lv(temp):
    temp_c = temp - 273.12
    Lv = 2.510e6 - 2359*temp_c
    return Lv

Lv = comp_Lv(T)
cp = 1003
hflux = cp*T - Lv*q

In [16]:
# take zonal means:
u_bar = u.mean('longitude')
v_bar = v.mean('longitude')
w_bar = w.mean('longitude')
T_bar = T.mean('longitude')
theta_bar = theta.mean('longitude')
hflux_bar = hflux.mean('longitude')

# get primes
v_prime = v - v_bar
u_prime = u - u_bar
w_prime = w - w_bar
T_prime = T - T_bar
theta_prime = theta - theta_bar
hflux_prime = hflux - hflux_bar

compute the following:
- EKE $[u'^2 + v'^2]/2$
- vertical wave heat flux: $[w'T']$
- eddy momentum flux: $[u'v']$
- meridional wave heat flux: $[v'T']$
- zonal mean KE: $[u]^2/2$
- meridional EP flux: $-\rho \cos (\varphi) [u'v']$
    - we can use a reference density profile of $\rho = e^{-z/H}$ where $H = 6.8 km$ and is the scale height
- vertical EP flux: $f\rho a \cos (\varphi) [u'\theta']/[\theta]_z$
    - we will use $6,371 km$ as the raidius of the earth
    - recall that the formula to calculate potential temperature is: $\theta = (p_0/p)^{R/c_p} T$, and we estimate $R/c_p = 0.286$.
- total heat flux: $c_p T + L_v q$ 
    - to estimate $L$ we will use the relation in chapter 18, appendix A of Vallis so that $L_v(T) = (2.501 \times 10^6 - 2359 T) J kg^{-1}$ where $T$ is in Celsius.
    - We will use $c_p =  1003 J kg^{-1} K^{-1}$
- CAPE
- the fourier components, is there always a sharp/"unique" time scale at ~25 days?

In [17]:
# compute some other quantities
EKE = ((u_prime**2 + v_prime**2)/2)
wpTp = (w_prime*T_prime)
upvp = (u_prime*v_prime)
zonalKE = u_bar**2/2
vpTp = v_prime*T_prime
vphfluxp = v_prime*hflux_prime

a = 6.37e6
Omega = 7.292e-5
f = 2 * Omega * np.sin(lat)

meridEPflux = -rho*np.cos(lat)*upvp
upthetap = (u_prime*theta_prime)
vertEPflux = f*rho*a*np.cos(lat)*upthetap/(theta_bar.mean('z'))

In [18]:
newds = xr.Dataset({
    'u' : u,
    'v' : v,
    'EKE' : EKE,
    'KE_zonal' : zonalKE,
    'wpTp' : wpTp,
    'upvp' : upvp,
    'EP_merid' : meridEPflux,
    'EP_vert' :  vertEPflux,
    'cape' : cape,
    'vpTp' : vpTp,
    'vpheatp': vphfluxp}).mean('longitude').rename({'z' : 'pressure'})

# convert back to original height and lat coords
p = (p_surf * np.exp(-newds['pressure'] / H) / 100).assign_attrs(units='hPa')
lat = (180 * newds['latitude'] / np.pi).assign_attrs(units='degrees_north')
newds = newds.assign_coords(pressure=p, latitude=lat).drop('z')

In [19]:
display(newds)

<xarray.Dataset>
Dimensions:   (latitude: 721, pressure: 37, time: 516)
Coordinates:
  * latitude  (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time      (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2021-12-01
  * pressure  (pressure) float64 1.0 2.0 3.0 5.0 7.0 ... 925.0 950.0 975.0 1e+03
Data variables:
    u         (time, pressure, latitude) float32 dask.array<shape=(516, 37, 721), chunksize=(12, 37, 721)>
    v         (time, pressure, latitude) float32 dask.array<shape=(516, 37, 721), chunksize=(12, 37, 721)>
    EKE       (time, pressure, latitude) float32 dask.array<shape=(516, 37, 721), chunksize=(12, 37, 721)>
    KE_zonal  (time, pressure, latitude) float32 dask.array<shape=(516, 37, 721), chunksize=(12, 37, 721)>
    wpTp      (time, pressure, latitude) float32 dask.array<shape=(516, 37, 721), chunksize=(12, 37, 721)>
    upvp      (time, pressure, latitude) float32 dask.array<shape=(516, 37, 721), chunksize=(12, 37, 721)>
    EP_merid  (pressure, 

note that the saving to netcdf is where the computations are actually completed, it is not particularly fast (a few hours) and takes a substantial amount of memory. (I believe about 115 GB at most.) However, the total file size of the completed file is less than a GB which is ok.

In [20]:
newds.to_netcdf("/glade/u/home/cvalva/atmoclass/eddy_data.nc")

In [ ]:
print("completed saving and processing") 